# TENSORFLOW

In [1]:
# Google Drive API
from google.colab import drive

# Drive bağlantısını oluştur
drive.mount('/content/gdrive')


Mounted at /content/gdrive


In [3]:
import os
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import layers
from tensorflow.keras import regularizers

# Veri klasörlerini belirt
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Okralı/Absorbance"
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Sağlıklı/Absorbance"

# Verileri depolamak için boş bir DataFrame oluştur
tum_veriler = pd.DataFrame()

# .csv dosyalarını oku ve NaN değerleri ortalama ile doldur
for klasor, label in [(okra_klasoru, 'Okra'), (normal_klasoru, 'Normal')]:
    dosyalar = os.listdir(klasor)

    for dosya in dosyalar:
        dosya_yolu = os.path.join(klasor, dosya)
        veri = pd.read_csv(dosya_yolu)
      #  veri = veri.fillna(veri.mean())  # NaN değerleri ortalama ile doldur
        veri['Label'] = label  # Etiket sütunu ekle
        tum_veriler = pd.concat([tum_veriler, veri], ignore_index=True)

# Verileri sınıflandırma için hazırla
X = tum_veriler[['Wavelength (nm)','Absorbance (AU)']]
y = tum_veriler['Label']

# Etiketleri tamsayı değerlerine dönüştür
y = y.replace({'Normal': 0, 'Okra': 1})

# Veri normalizasyonu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Verileri eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# TensorFlow modelini tanımla
model = tf.keras.Sequential([
    layers.Input(shape=X_train.shape[1:]),  # Giriş katmanı
    layers.Dense(256, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # Gizli katman
    layers.Dropout(0.5),  # Dropout katmanı
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),  # Gizli katman
    layers.Dropout(0.5),  # Dropout katmanı
    layers.Dense(1, activation='sigmoid')  # Çıkış katmanı (ikili sınıflandırma)
])

# Modeli derle
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Modeli eğit
model.fit(X_train, y_train, epochs=30, batch_size=32, validation_split=0.2)

# Modeli değerlendir
loss, accuracy = model.evaluate(X_test, y_test)
print("Tensorflow Test Kaybı:", loss)
print("Tensorflow Test Doğruluğu:", accuracy)


Epoch 1/30
4560/4560 [==============================] - 15s 3ms/step - loss: 0.6651 - accuracy: 0.6364 - val_loss: 0.6480 - val_accuracy: 0.6490
Epoch 2/30
4560/4560 [==============================] - 16s 3ms/step - loss: 0.6510 - accuracy: 0.6442 - val_loss: 0.6451 - val_accuracy: 0.6484
Epoch 3/30
4560/4560 [==============================] - 12s 3ms/step - loss: 0.6499 - accuracy: 0.6446 - val_loss: 0.6434 - val_accuracy: 0.6489
Epoch 4/30
4560/4560 [==============================] - 12s 3ms/step - loss: 0.6493 - accuracy: 0.6443 - val_loss: 0.6445 - val_accuracy: 0.6462
Epoch 5/30
4560/4560 [==============================] - 13s 3ms/step - loss: 0.6489 - accuracy: 0.6448 - val_loss: 0.6467 - val_accuracy: 0.6472
Epoch 6/30
4560/4560 [==============================] - 12s 3ms/step - loss: 0.6493 - accuracy: 0.6430 - val_loss: 0.6439 - val_accuracy: 0.6470
Epoch 7/30
4560/4560 [==============================] - 12s 3ms/step - loss: 0.6483 - accuracy: 0.6438 - val_loss: 0.6442 - val_ac

# RNN

In [5]:
import os
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Veri klasörlerini belirt
okra_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Okralı/Absorbance"
normal_klasoru = "/content/gdrive/MyDrive/AliBaki_TURKOZ_Engineering_Project/Data_Sets/Raisin/Sağlıklı/Absorbance"

# Verileri depolamak için boş bir DataFrame oluştur
tum_veriler = pd.DataFrame()

# .csv dosyalarını oku ve DataFrame'e ekle
for klasor, label in [(okra_klasoru, 'Okra'), (normal_klasoru, 'Normal')]:
    dosyalar = os.listdir(klasor)
    for dosya in dosyalar:
        dosya_yolu = os.path.join(klasor, dosya)
        veri = pd.read_csv(dosya_yolu)
      #  veri = veri.fillna(veri.mean())  # NaN değerleri ortalama ile doldur
        veri['Label'] = label  # Etiket sütunu ekle
        tum_veriler = pd.concat([tum_veriler, veri], ignore_index=True)

# Verileri sınıflandırma için hazırla
X = tum_veriler[['Wavelength (nm)','Absorbance (AU)']]
y = tum_veriler['Label']

# Veri normalizasyonu
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Verileri eğitim ve test setlerine ayır
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

# Label encoding for y
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)

# Reshape the input data for RNN
X_train_rnn = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
X_test_rnn = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])

# Build the RNN model
model = Sequential([
    SimpleRNN(64, input_shape=(X_train_rnn.shape[1], X_train_rnn.shape[2])),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
history = model.fit(X_train_rnn, y_train_encoded, epochs=20, batch_size=32, validation_split=0.2)

# Evaluate the model
_, accuracy = model.evaluate(X_test_rnn, y_test_encoded)
print("RNN Doğruluk Oranı:", accuracy)


Epoch 1/20
4560/4560 [==============================] - 14s 3ms/step - loss: 0.6398 - accuracy: 0.6444 - val_loss: 0.6350 - val_accuracy: 0.6454
Epoch 2/20
4560/4560 [==============================] - 13s 3ms/step - loss: 0.6314 - accuracy: 0.6516 - val_loss: 0.6295 - val_accuracy: 0.6513
Epoch 3/20
4560/4560 [==============================] - 12s 3ms/step - loss: 0.6300 - accuracy: 0.6519 - val_loss: 0.6315 - val_accuracy: 0.6476
Epoch 4/20
4560/4560 [==============================] - 13s 3ms/step - loss: 0.6289 - accuracy: 0.6521 - val_loss: 0.6348 - val_accuracy: 0.6487
Epoch 5/20
4560/4560 [==============================] - 13s 3ms/step - loss: 0.6286 - accuracy: 0.6519 - val_loss: 0.6291 - val_accuracy: 0.6501
Epoch 6/20
4560/4560 [==============================] - 13s 3ms/step - loss: 0.6282 - accuracy: 0.6527 - val_loss: 0.6291 - val_accuracy: 0.6523
Epoch 7/20
4560/4560 [==============================] - 13s 3ms/step - loss: 0.6281 - accuracy: 0.6516 - val_loss: 0.6286 - val_ac